# Householder QR

The numerical downfall of the Gram-Schmidt algorithm lies in performing multiple operations and hoping to end up with a unitary resulting $Q$. The key to a stable algorithm is to go the other way: operate orthogonally (unitarily) on $A$ in order to produce a triangular $R$. It's a lot like Gaussian elimination, proceeding one column at a time. Instead of elementary row operations, though, we want orthogonal transformations. Two natural sources of these are reflections and rotations. For general matrices, the reflection approach is more efficient. 

The key step is to find, given $x$, a unitary $F$ such that $Fx=\alpha e_1$ for a scalar $\alpha$. Since $F$ preserves the 2-norm, $\alpha= \pm \|x\|_2$. One can simply exhibit the solution. Define $v=\alpha e_1 - x$ and then set $$ F = I - 2 \frac{vv^*}{v^*v}.$$

In [1]:
using LinearAlgebra
x = randn(5);  alpha = norm(x);
v = alpha*[1;zeros(4)] - x;
F = I - 2*(v*v')/dot(v,v)

5×5 Array{Float64,2}:
 -0.179924   -0.43483     0.423623    0.0621187   0.771515 
 -0.43483     0.839755    0.156115    0.0228922   0.284321 
  0.423623    0.156115    0.847909   -0.0223022  -0.276993 
  0.0621187   0.0228922  -0.0223022   0.99673    -0.0406174
  0.771515    0.284321   -0.276993   -0.0406174   0.495531 

In [2]:
F'*F-I

5×5 Array{Float64,2}:
  4.44089e-16   1.55849e-16  -1.44617e-16  -1.03932e-17  -3.32121e-16
  1.55849e-16   2.22045e-16  -8.31031e-17  -7.03507e-18  -2.03525e-17
 -1.44617e-16  -8.31031e-17   0.0           1.36215e-17   9.5301e-17 
 -1.03932e-17  -7.03507e-18   1.36215e-17   0.0           2.24549e-17
 -3.32121e-16  -2.03525e-17   9.5301e-17    2.24549e-17   0.0        

In [3]:
F*x

5-element Array{Float64,1}:
  1.7662873565740382    
  5.551115123125783e-17 
 -1.6653345369377348e-16
 -6.938893903907228e-18 
 -2.220446049250313e-16 

So we proceed through columns $j=1,2,\ldots$ of $A$. In column $j$, we want to introduce zeros below the diagonal, so $x$ is drawn from rows $j$ to $m$. For example, with $j=1$ (note that $v$ is normalized to simplify the formula for the reflector):

In [4]:
A = rand(0.:5.,6,3);  
v = -A[:,1];  # -x
v[1] += norm(v);
v = v/norm(v);
F = I - 2*(v*v');
A = F*A

6×3 Array{Float64,2}:
  6.08276       4.27437   4.43877 
 -7.77156e-16  -1.10452  -0.421337
  0.0           0.0       1.0     
 -7.77156e-16  -2.10452   1.57866 
 -7.77156e-16  -2.10452   1.57866 
 -8.88178e-16  -2.15678   0.367994

Now we can proceed with $j=2$. 

In [5]:
v = -A[2:end,2];  # -x
v[1] += norm(v);
v = v/norm(v);
F = I - 2*(v*v');
A[2:end,2:end] = F*A[2:end,2:end];
A

6×3 Array{Float64,2}:
  6.08276       4.27437       4.43877 
 -7.77156e-16   3.83793      -1.81686 
  0.0           0.0           1.0     
 -7.77156e-16  -3.82601e-16   0.984444
 -7.77156e-16  -2.71578e-16   0.984444
 -8.88178e-16  -3.82061e-16  -0.240981

And so on. Once $j$ sweeps from 1 to $n$, the matrix will be transformed into the $R$ we seek. If we properly accumulate the actions of the reflectors, we end up with $Q$ as well (the full one or the thin one, as we choose).  

I have left out a detail that is important to the stability. A full code is part of the exercises. 

## Q-less QR

When we solved the linear system $Ax=b$ with the factorization, we transformed it to the equivalent system $Rx=Q^*b$. Thus (as is often the case) we don't really want $Q$, or even $Q^*$, but the ability to compute $Q^*b$ for given $b$. The Householder method computes $F_n\cdots F_1A=R,$ where each $F_j$ contains a reflector, so $$Q^*b=F_n^*\cdots F_1^*b=F_n\cdots F_1b.$$ That is, we just have to apply the reflectors to $b$ (in rows $j$ to $m$ for $F_j$). This is more efficient than constructing the $Q$ itself (which can be obtained by letting $b$ take on the columns of an identity matrix).  

This observation is why the built-in `qr` function does not really compute or store $Q$, but ([more or less](https://docs.julialang.org/en/stable/stdlib/LinearAlgebra/#LinearAlgebra.QRCompactWY)) the vectors that define the Householder reflectors. 